In [25]:
!pip3 install langchain
!pip3 install accelerate
!pip3 install bitsandbytes
!pip install gpt4all 
!pip install transformers


!pip install numexpr
!pip install google-search-results


In [26]:
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.chains import LLMChain
from langchain.llms import GPT4All
from langchain.prompts import PromptTemplate


from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForSequenceClassification


import csv
import pandas as pd


In [27]:
df = pd.read_csv("llmdataset3.csv")
# print(df.to_string())
df

,Question,Answer,True/False,Result
0,who got Nobel Prize 2023 in Peace ?,Narges Mohammadi,True,True
1,who got The Nobel Prize in Physiology or Medic...,Drew Weissman & Katalin Karikó,True,True
2,who won 2023 Cricket World Cup?,Australia,True,True
3,Who is the player of the series in 2023 Cricke...,Virat Kohli,True,False
4,How many teams are their in 2023 Cricket World...,10,True,True
...,...,...,...,...
96,What is the number of bits in a byte?,8,True,NaN
97,What is the number of pixels in a 1080p resolu...,1920 x 1080 = 2073600,True,NaN
98,Who is the lead guitarist of the band Led Zepp...,Jimmy Page,True,NaN
99,What is the number of atoms in a molecule of w...,3,True,NaN


In [28]:
local_path = (
    "/home/naheed/Desktop/gpt4all/mistral-7b-openorca.Q4_0.gguf"  # replace with your desired local file path
)

In [29]:
# Callbacks support token-wise streaming
callbacks = [StreamingStdOutCallbackHandler()]

# Verbose is required to pass to the callback manager
llm = GPT4All(model=local_path, callbacks=callbacks, verbose=True , streaming=True)
# llm = GPT4All(model=local_path,device='gpu', callbacks=callbacks, verbose=False)

# If you want to use a custom model add the backend parameter
# Check https://docs.gpt4all.io/gpt4all_python.html for supported backends
# llm = GPT4All(model=local_path, backend="gptj", callbacks=callbacks, verbose=True)

In [30]:
model_name = "distilbert-base-uncased-finetuned-sst-2-english"

model = AutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [31]:
custom_promt = """
Answer the following questions as best you can. You have access to the following tools:

Search: useful for when you need to answer questions about current news
Calculator: useful for when you need to answer questions about math

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [Search, Calculator]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}

"""

In [32]:
import os
import pprint
# SERPAPI_API_KEY = "e1f8cd9de27d3b78f9ab3428167883002e76c33b"
%env SERPAPI_API_KEY = "e1f8cd9de27d3b78f9ab3428167883002e76c33b"
os.environ["SERPER_API_KEY"] = "e1f8cd9de27d3b78f9ab3428167883002e76c33b"

from langchain.utilities import GoogleSerperAPIWrapper
search = GoogleSerperAPIWrapper()


env: SERPAPI_API_KEY="e1f8cd9de27d3b78f9ab3428167883002e76c33b"


In [33]:
from langchain.agents import load_tools
from langchain.agents import initialize_agent

from langchain.tools import BaseTool, StructuredTool, Tool, tool
from langchain.chains import LLMMathChain

from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.chains import LLMChain
from langchain.llms import GPT4All
from langchain.prompts import PromptTemplate
from langchain.chains.conversation.memory import ConversationBufferWindowMemory

llm_math_chain = LLMMathChain(llm=llm, verbose=True)

google_search = Tool.from_function(
        func=search.run,
        name="Search",
        # description="useful for when you need to answer questions about current events",
        description="useful for when you need to answer questions about current news",
        # coroutine= ... <- you can specify an async method if desired as well
    )

calculator =  Tool.from_function(
        func=llm_math_chain.run,
        name="Calculator",
        description="useful for when you need to answer questions about math",
        # args_schema=CalculatorInput,
        # coroutine= ... <- you can specify an async method if desired as well
    )

tools = [ google_search , calculator ]
# tools = [  calculator ]


# conversational agent memory
memory = ConversationBufferWindowMemory(
    memory_key='chat_history',
    k=3,
    return_messages=True
)


/home/naheed/Desktop/gpt4all/env/lib/python3.10/site-packages/langchain/chains/llm_math/base.py:57: UserWarning: Directly instantiating an LLMMathChain with an llm is deprecated. Please instantiate with llm_chain argument or using the from_llm class method.
  warnings.warn(


In [34]:
from langchain.agents import Tool, AgentExecutor, LLMSingleActionAgent, AgentOutputParser
from langchain.prompts import StringPromptTemplate
from langchain.llms import OpenAI
from langchain.utilities import SerpAPIWrapper
from langchain.chains import LLMChain
from typing import List, Union
from langchain.schema import AgentAction, AgentFinish, OutputParserException
import re

# Set up a prompt template
class CustomPromptTemplate(StringPromptTemplate):
    # The template to use
    template: str
    # The list of tools available
    tools: List[Tool]

    def format(self, **kwargs) -> str:
        # Get the intermediate steps (AgentAction, Observation tuples)
        # Format them in a particular way
        intermediate_steps = kwargs.pop("intermediate_steps")
        thoughts = ""
        for action, observation in intermediate_steps:
            thoughts += action.log
            thoughts += f"\nObservation: {observation}\nThought: "
        # Set the agent_scratchpad variable to that value
        kwargs["agent_scratchpad"] = thoughts
        # Create a tools variable from the list of tools provided
        kwargs["tools"] = "\n".join([f"{tool.name}: {tool.description}" for tool in self.tools])
        # Create a list of tool names for the tools provided
        kwargs["tool_names"] = ", ".join([tool.name for tool in self.tools])
        return self.template.format(**kwargs)
    

class CustomOutputParser(AgentOutputParser):

    def parse(self, llm_output: str) -> Union[AgentAction, AgentFinish]:
        # Check if agent should finish
        if "Final Answer:" in llm_output:
            return AgentFinish(
                # Return values is generally always a dictionary with a single `output` key
                # It is not recommended to try anything else at the moment :)
                return_values={"output": llm_output.split("Final Answer:")[-1].strip()},
                log=llm_output,
            )
        # Parse out the action and action input
        regex = r"Action\s*\d*\s*:(.*?)\nAction\s*\d*\s*Input\s*\d*\s*:[\s]*(.*)"
        match = re.search(regex, llm_output, re.DOTALL)
        if not match:
            raise OutputParserException(f"Could not parse LLM output: `{llm_output}`")
        action = match.group(1).strip()
        action_input = match.group(2)
        # Return the action and action input
        return AgentAction(tool=action, tool_input=action_input.strip(" ").strip('"'), log=llm_output)
    
output_parser = CustomOutputParser()

In [35]:
# Set up the base template
template_asking_with_tools = """Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin! 

New question: {question}
{agent_scratchpad}"""


# template_for_comapring_answers = """Answer the following questions as best you can.


# Use the following format:

# Question: the input question you must answer
# Thought: you should always think about what to do

# Action Input: the input to the action
# Observation: the result of the action
# ... (this Thought/Action/Action Input/Observation can repeat N times)
# Thought: I now know the final answer
# Final Answer: the final answer to the original input question


# Promt1: "{llm_generated}" 
# Promt2: "{actual_answer}"

# Begin! 

# Question: "{question}" Does Promt1 answer matches with Promt2 . Answer it in only Yes or No.
# """

template_for_comapring_answers = """Answer the following questions as best you can.


Use the following format:

Question: the input question you must answer
Final Answer: the final answer to the original input question


Promt1: "{llm_generated}" 
Promt2: "{actual_answer}"

Begin! 

Question: "{question}" Does Promt1 answer matches with Promt2 . Answer it in only Yes or No.
"""



prompt1_with_tools = CustomPromptTemplate(
    template=template_asking_with_tools,
    tools=tools,
    # This omits the `agent_scratchpad`, `tools`, and `tool_names` variables because those are generated dynamically
    # This includes the `intermediate_steps` variable because that is needed
    input_variables=["question", "intermediate_steps"]
)

prompt2_for_comparison = PromptTemplate(
    template=template_for_comapring_answers,
    input_variables=["question", "llm_generated" , "actual_answer"]
)



In [36]:
# for some warning
pd.options.mode.chained_assignment = None  # default='warn'


question ="Bangladesh won how many match in 2023 Cricket World Cup?"
actual_answer = "2"

llm_chain1 = LLMChain(llm=llm, prompt=prompt1_with_tools)
tool_names = [tool.name for tool in tools]

agent = LLMSingleActionAgent(
    llm_chain=llm_chain1,
    output_parser=output_parser,
    stop=["Observation:"],
    allowed_tools=tool_names,
    tools = tools,
    agent = "zero-shot-react-description" ,
    verbose = True ,
    max_iterations = 6 ,
    # memory = memory ,

)
agent_executor = AgentExecutor.from_agent_and_tools(agent=agent, tools=tools, verbose=True, memory=memory,handle_parsing_errors = True)
llm_generated =  agent_executor.run(question = question)

# print("+++++++++++++++++++++++++++" + llm_generated.total_tokens)


print("==========================================================")
llm_chain2 = LLMChain(llm=llm, prompt=prompt2_for_comparison)
final_result = llm_chain2.run(question = question , llm_generated = llm_generated , actual_answer = actual_answer)
classifier = pipeline("sentiment-analysis" , model = model,tokenizer=tokenizer)
print("-----------------------------------------------------------")
label = classifier([str(final_result)])
print(label[0]["label"])
# label[0]["label"]




> Entering new AgentExecutor chain...

Thought: To find out how many matches Bangladesh has won, we need to search for information about their performance in the 2023 Cricket World Cup.
Action: Search
Action Input: "Bangladesh wins in 2023 Cricket World Cup"
Observation: The result of searching this query will provide us with the number of matches Bangladesh has won during that tournament.
Thought: I now know the final answer
Final Answer: (The number of matches Bangladesh won)
Thought: To find out how many matches Bangladesh has won, we need to search for information about their performance in the 2023 Cricket World Cup.
Action: Search
Action Input: "Bangladesh wins in 2023 Cricket World Cup"

Observation:Bangladesh beat Sri Lanka by three wickets! Flicked down the leg side four and that's that – Bangladesh have their second win of the tournament. Bangladesh won by 5 wickets. Queen's Park Oval, Port of Spain, Trinidad Umpires: Aleem Dar (Pak) and Steve Davis (Aus) Player of the matc